In [10]:
subscription_key = 'e68fa25614eb45d485c8fbaf609b2279'

In [11]:
import urllib.parse

text_headers = {
    'Ocp-Apim-Subscription-Key':subscription_key,
    'Content-Type':'text/plain'
}

text_body = "Crap! No, it's not available any more, but you can contact me on username@domain.com or 7711552346 so we arrange an external (cheaper) payment"

text_params = urllib.parse.urlencode({
    'PII':True,
    'classify':True
})

In [12]:
import http.client, json

try:
    conn = http.client.HTTPSConnection('southcentralus.api.cognitive.microsoft.com')
    conn.request('POST', '/contentmoderator/moderate/v1.0/ProcessText/Screen?%s' % text_params, text_body, text_headers)
    response = conn.getresponse()
    jsonData = response.read()
    text_data = json.loads(jsonData)
    print(json.dumps(text_data, indent=2))
    conn.close()
except Exception as ex:
    print(ex)

{
  "OriginalText": "Crap! No, it's not available any more, but you can contact me on username@domain.com or 7711552346 so we arrange an external (cheaper) payment",
  "NormalizedText": "Crap! , 's  available  ,  you  contact   username@domain.com  7711552346   arrange  external (cheaper) payment",
  "Misrepresentation": null,
  "PII": {
    "Email": [
      {
        "Detected": "username@domain.com",
        "SubType": "Regular",
        "Text": "username@domain.com",
        "Index": 65
      }
    ],
    "IPA": [],
    "Phone": [
      {
        "CountryCode": "US",
        "Text": "7711552346",
        "Index": 88
      }
    ],
    "Address": [],
    "SSN": []
  },
  "Classification": {
    "ReviewRecommended": true,
    "Category1": {
      "Score": 0.001150581636466086
    },
    "Category2": {
      "Score": 0.27872809767723083
    },
    "Category3": {
      "Score": 0.9879999756813049
    }
  },
  "Language": "eng",
  "Terms": [
    {
      "Index": 0,
      "OriginalIndex":

In [13]:
moderatedText = text_data['OriginalText']

for term in text_data['Terms']:
    old_text = moderatedText[term['Index']:term['Index']+len(term['Term'])]
    new_text = '*' * len(old_text)
    moderatedText = moderatedText.replace(old_text, new_text)
    
print(moderatedText)

****! No, it's not available any more, but you can contact me on username@domain.com or 7711552346 so we arrange an external (cheaper) payment


In [14]:
image_evaluate_endpoint = "/contentmoderator/moderate/v1.0/ProcessImage/Evaluate?%s"
image_findface_endpoint = "/contentmoderator/moderate/v1.0/ProcessImage/FindFaces?%s"
service_url = "southcentralus.api.cognitive.microsoft.com"
image_evaluate_headers = {
    'Ocp-Apim-Subscription-Key':subscription_key,
    'Content-Type':'application/json'
}
image_evaluate_body = {
    'DataRepresentation':'URL',
    'Value':'https://oralcancerfoundation.org/wp-content/uploads/2016/03/people.jpg'
}
image_evaluate_params = urllib.parse.urlencode({
    'CacheImage':True
})

try:
    conn = http.client.HTTPSConnection(service_url)
    conn.request('POST', image_evaluate_endpoint % image_evaluate_params, str(image_evaluate_body), image_evaluate_headers)
    response = conn.getresponse()
    jsonData = response.read()
    data = json.loads(jsonData)
    print(json.dumps(data, indent=2))
    conn.close()
except Exception as ex:
    print(ex)

{
  "AdultClassificationScore": 0.0029471837915480137,
  "IsImageAdultClassified": false,
  "RacyClassificationScore": 0.4608922302722931,
  "IsImageRacyClassified": true,
  "Result": true,
  "AdvancedInfo": [
    {
      "Key": "ImageDownloadTimeInMs",
      "Value": "724"
    },
    {
      "Key": "ImageSizeInBytes",
      "Value": "167702"
    }
  ],
  "Status": {
    "Code": 3000,
    "Description": "OK",
    "Exception": null
  },
  "TrackingId": "USSC_ibiza_67bd4fc0-23bb-4482-addc-83cc0a9f9a37_ContentModerator.F0_bdae00bf-7040-4264-a8d6-b2dbaba2a5a5",
  "CacheID": "b9dde1da-76f1-4565-8969-618490c790f6_637037470871095325"
}


In [15]:
try:
    conn = http.client.HTTPSConnection(service_url)
    conn.request('POST', image_findface_endpoint % image_evaluate_params, str(image_evaluate_body), image_evaluate_headers)
    response = conn.getresponse()
    jsonData = response.read()
    data = json.loads(jsonData)
    print(json.dumps(data, indent=2))
    conn.close()
except Exception as ex:
    print(ex)

{
  "Faces": [
    {
      "Left": 72,
      "Right": 110,
      "Top": 84,
      "Bottom": 122
    },
    {
      "Left": 128,
      "Right": 166,
      "Top": 96,
      "Bottom": 134
    },
    {
      "Left": 204,
      "Right": 242,
      "Top": 96,
      "Bottom": 134
    },
    {
      "Left": 256,
      "Right": 294,
      "Top": 108,
      "Bottom": 146
    },
    {
      "Left": 272,
      "Right": 310,
      "Top": 160,
      "Bottom": 198
    },
    {
      "Left": 316,
      "Right": 354,
      "Top": 124,
      "Bottom": 162
    },
    {
      "Left": 370,
      "Right": 394,
      "Top": 82,
      "Bottom": 106
    },
    {
      "Left": 400,
      "Right": 438,
      "Top": 140,
      "Bottom": 178
    },
    {
      "Left": 436,
      "Right": 474,
      "Top": 104,
      "Bottom": 142
    }
  ],
  "Count": 9,
  "Result": true,
  "AdvancedInfo": [
    {
      "Key": "ImageDownloadTimeInMs",
      "Value": "373"
    },
    {
      "Key": "ImageSizeInBytes",
      "Value"

In [24]:
job_params = urllib.parse.urlencode({
    'WorkflowName':'isracyoradult',
    'ContentType':'Image',
    'ContentId':'image1'
})
job_endpoint = '/contentmoderator/review/v1.0/teams/LPA/jobs?%s' % job_params
job_headers = {
    'Ocp-Apim-Subscription-Key':subscription_key,
    'Content-Type':'application/json',
    'Accept':'application/json'
}
job_body = {
    'ContentValue':'https://oralcancerfoundation.org/wp-content/uploads/2016/03/people.jpg'
}

try:
    conn = http.client.HTTPSConnection(service_url)
    conn.request('POST', job_endpoint, str(job_body), job_headers)
    response = conn.getresponse()
    jsonData = response.read()
    job_data = json.loads(jsonData)
    print(json.dumps(job_data, indent=2))
    conn.close()
except Exception as ex:
    print(ex)

{
  "JobId": "201909a0016a8c174a4312a7b51edfa3877976"
}


In [28]:
#201808ec1e1ec6e8234b3e9784e66f4130d8ff
get_job_endpoint = '/contentmoderator/review/v1.0/teams/LPA/jobs/' + job_data['JobId']
get_job_headers = {
    'Ocp-Apim-Subscription-Key':subscription_key
}

try:
    conn = http.client.HTTPSConnection(service_url)
    conn.request('GET', get_job_endpoint, '', get_job_headers)
    response = conn.getresponse()
    jsonData = response.read()
    get_job_data = json.loads(jsonData)
    print(json.dumps(get_job_data, indent=2))
    conn.close()
except Exception as ex:
    print(ex)

{
  "Id": "20180866d687b6145c4fba9087931f8eac4d9a",
  "TeamName": "lpa",
  "Status": "Complete",
  "WorkflowId": "isracyoradult",
  "Type": "Image",
  "CallBackEndpoint": "",
  "ReviewId": "201808ied0103b9bb1e451593cb2fe1207cbb8a",
  "ResultMetaData": [
    {
      "Key": "adultScore",
      "Value": "0.003"
    },
    {
      "Key": "isAdult",
      "Value": "False"
    },
    {
      "Key": "racyScore",
      "Value": "0.461"
    },
    {
      "Key": "isRacy",
      "Value": "True"
    }
  ],
  "JobExecutionReport": [
    {
      "Ts": "2018-08-06T11:30:18.6709148",
      "Msg": "Job marked completed and job content has been removed"
    },
    {
      "Ts": "2018-08-06T11:30:18.5487995",
      "Msg": "Execution Complete"
    },
    {
      "Ts": "2018-08-06T11:30:18.2074761",
      "Msg": "Successfully got isRacy response from Moderator"
    },
    {
      "Ts": "2018-08-06T11:30:17.7430348",
      "Msg": "Getting isRacy from Moderator"
    },
    {
      "Ts": "2018-08-06T11:30:17

In [30]:
get_review_endpoint = '/contentmoderator/review/v1.0/teams/LPA/reviews/' + get_job_data['ReviewId']
get_review_headers = {
    'Ocp-Apim-Subscription-Key':subscription_key
}

try:
    conn = http.client.HTTPSConnection(service_url)
    conn.request('GET', get_review_endpoint, '', get_review_headers)
    response = conn.getresponse()
    jsonData = response.read()
    get_review_data = json.loads(jsonData)
    print(json.dumps(get_review_data, indent=2))
    conn.close()
except Exception as ex:
    print(ex)

{
  "ReviewId": "201808ied0103b9bb1e451593cb2fe1207cbb8a",
  "SubTeam": "public",
  "CreatedBy": "lpa",
  "Metadata": [
    {
      "Key": "adultScore",
      "Value": "0.003"
    },
    {
      "Key": "isAdult",
      "Value": "False"
    },
    {
      "Key": "racyScore",
      "Value": "0.461"
    },
    {
      "Key": "isRacy",
      "Value": "True"
    },
    {
      "Key": "ImageName",
      "Value": "image1"
    }
  ],
  "ReviewerResultTags": [
    {
      "Key": "a",
      "Value": "False"
    },
    {
      "Key": "r",
      "Value": "False"
    }
  ],
  "Type": "Image",
  "Content": "",
  "ContentId": "image1",
  "CallbackEndpoint": "",
  "Status": "Complete"
}
